##### This is my training project to learn LangGraph agents, the idea is to create an Agent with different tools starting from a basic agent to a full agent with tools and memory, then ask it different things and check if is using the tools and the process.


In [ ]:
%pip install -U langgraph 

## Libraries

In [5]:
# Load variables
from dotenv import load_dotenv 
load_dotenv()
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

## StateGraph

In [ ]:
# SatateGraph Creation
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

## Define the model


In [2]:
# Define the model
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

## Define chatbot node

In [ ]:
# Define chatbot Node, a node in LangGraph is tipically a regular Python function
def chatbot(state: State):
    return {"messages": [model.invoke(state["messages"])]}

# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot) # The first argument is the unique node name, the second is the function that will be called
# whenever the node is used.

## Entry and exit point

In [ ]:
# Add an entry and exit point
graph_builder.add_node(START, "chatbot")
graph_builder.add_node("chatbot", END)

## Compile the graph

In [ ]:
graph = graph_builder.compile()

## Visualize the graph

In [ ]:
# LangGraph allows to visualize the graph with the get_graph method
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass